# LangGraph Basics: Understanding Nodes, Edges, and State

This Colab notebook introduces the fundamental concepts of LangGraph: nodes, edges, and state. We'll use a simple sequential graph to illustrate these concepts.



## Setup

First, let's install the required package:

In [ ]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


Now, let's import the necessary modules:


In [ ]:
import datetime
import operator
import time
from typing import Annotated, Any, Dict, List
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

## Understanding State in LangGraph

In LangGraph, the state represents the data that flows through the graph. It's defined using a TypedDict:


In [ ]:
class State(TypedDict):
    aggregate: Annotated[list, operator.add]


Here's what this means:
- `State` is a TypedDict, which allows us to specify the structure of our state.
- `aggregate` is a key in our state, which will hold a list.
- `Annotated[list, operator.add]` specifies that when the state is updated, lists should be combined using the `add` operation (which for lists means concatenation).

The state is passed between nodes and can be modified by each node in the graph.

## Understanding Nodes in LangGraph

Nodes in LangGraph represent the processing steps in your workflow. Each node is a function that takes the current state as input and returns updates to the state.

Let's look at one of our node functions:


In [ ]:
def hello(state: State) -> Dict[str, Any]:
    print("#### hello enter,")
    time.sleep(1)
    return {"aggregate": ["hello"]}


def welcome(state: State) -> Dict[str, Any]:
    print("#### welcome enter,")
    time.sleep(1)
    return {"aggregate": ["welcome"]}


def to(state: State) -> Dict[str, Any]:
    print("#### to enter,")
    time.sleep(1)
    return {"aggregate": ["to"]}


def odsc(state: State) -> Dict[str, Any]:
    print("#### odsc enter,")
    time.sleep(1)
    return {"aggregate": ["ODSC"]}



Key points about nodes:
1. They take the current `State` as an input.
2. Prints a message when it's entered
3. They return a dictionary with updates to the state.
3. In this example, each node adds its name to the `aggregate` list in the state.
4. The `time.sleep(1)` is just to simulate some work being done.




## Building the Graph
Let's start by creating our StateGraph:


In [ ]:
builder = StateGraph(State)


### Adding Nodes

Next, we add nodes to our graph:


In [ ]:
builder.add_node("a", hello)
builder.add_node("b", welcome)
builder.add_node("c", to)
builder.add_node("d", odsc)


Each `add_node` call does two things:
1. It assigns a unique identifier to the node (e.g., "a", "b", "c", "d").
2. It associates a function with that node (e.g., `hello`, `welcome`, `to`, `odsc`).

These functions are the ones we defined earlier, each representing a step in our workflow.

### Adding Edges

After adding nodes, we define how they connect to each other using edges:



In [ ]:
builder.add_edge(START, "a")
builder.add_edge("a", "b")
builder.add_edge("b", "c")
builder.add_edge("c", "d")
builder.add_edge("d", END)

Each `add_edge` call creates a directed connection between two nodes:
- `START` and `END` are special nodes representing the beginning and end of the graph.
- The first `add_edge` connects the start of the graph to node "a".
- The subsequent calls connect "a" to "b", "b" to "c", and so on.
- The last `add_edge` connects node "d" to the end of the graph.

This sequence of edges defines the flow of execution in our graph.

## Compiling the Graph

Finally, we compile the graph:

In [ ]:
graph = builder.compile()


This step finalizes the graph structure. After compilation:
- The graph is ready for execution.
- No further modifications can be made to the graph structure.
- We can now use methods like `invoke` or `stream` to run the graph.

In [ ]:
res = graph.invoke({"aggregate": []}, {"configurable": {"thread_id": "foo"}})
print(f"{res=}")

#### hello enter,
#### welcome enter,
#### to enter,
#### odsc enter,
res={'aggregate': ['hello', 'welcome', 'to', 'ODSC']}


In [ ]:
for state in graph.stream({"aggregate": []}, {"configurable": {"thread_id": "foo"}}, stream_mode="values"):
      print(f"{state=}")

state={'aggregate': []}
#### hello enter,
state={'aggregate': ['hello']}
#### welcome enter,
state={'aggregate': ['hello', 'welcome']}
#### to enter,
state={'aggregate': ['hello', 'welcome', 'to']}
#### odsc enter,
state={'aggregate': ['hello', 'welcome', 'to', 'ODSC']}
